<a href="https://colab.research.google.com/github/mattocanas/Antibody-Antigen-Interaction-Pred-CNN/blob/main/Antibody_Antigen_Interaction_Pred_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install biopython

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 26.3 MB/s eta 0:00:00


In [2]:
# Imports
import requests
from Bio import SeqIO
from Bio.PDB import PDBList, PDBParser, PPBuilder
import os
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Function to get sequences
def get_sequences_from_pdb(pdb_id, chain_id):
    pdb_id = pdb_id.lower()
    file_path = f"pdb{pdb_id}.ent"
    
    if not os.path.exists(file_path):
        try:
            pdbl = PDBList()
            pdbl.retrieve_pdb_file(pdb_id, pdir=".", file_format="pdb")
        except Exception as e:
            print(f"Error retrieving {pdb_id}: {e}")
            return None
    
    try:
        parser = PDBParser()
        structure = parser.get_structure(pdb_id, file_path)
    except Exception as e:
        print(f"Error parsing structure for {pdb_id}: {e}")
        return None

    ppb = PPBuilder()
    
    for model in structure:
        for chain in model:
            if chain.id == chain_id:
                for pp in ppb.build_peptides(chain):
                    sequence = pp.get_sequence()
                    return str(sequence)

    return None

In [5]:
dataset = []

with open("/content/drive/My Drive/positive samples.txt", "r") as f:
    for line in f:
        line = line.strip()
        
        # Try splitting by different delimiters
        if "\t" in line:
            antibody, antigen = line.split("\t")
        elif " " in line:
            antibody, antigen = line.split(" ")
        elif "," in line:
            antibody, antigen = line.split(",")
        else:
            print("Invalid line format:", line)
            continue

        dataset.append((antibody, antigen))

print(dataset)


Invalid line format: 
[('4k3j_H_L_B', '4k3j_H_L_B'), ('4ers_H_L_A', '4ers_H_L_A'), ('3tt1_I_M_A', '3tt1_I_M_A'), ('1qfu_H_L_A', '1qfu_H_L_A'), ('1qfu_H_L_A', '2vir_B_A_C'), ('1qfu_H_L_A', '4fp8_I_M_B'), ('1qfu_H_L_A', '4gms_J_N_E'), ('1qfu_H_L_A', '4gms_I_M_C'), ('1qfu_H_L_A', '5k9k_H_L_F'), ('1qfu_H_L_A', '5kaq_F_G_C'), ('1qfu_H_L_A', '6n5d_C_D_A'), ('1qfu_H_L_A', '6n5e_G_F_B'), ('1qfu_H_L_A', '6pdx_M_N_B'), ('2vir_B_A_C', '1qfu_H_L_A'), ('2vir_B_A_C', '2vir_B_A_C'), ('2vir_B_A_C', '4fp8_I_M_B'), ('2vir_B_A_C', '4gms_J_N_E'), ('2vir_B_A_C', '4gms_I_M_C'), ('2vir_B_A_C', '5k9k_H_L_F'), ('2vir_B_A_C', '5kaq_F_G_C'), ('2vir_B_A_C', '6n5d_C_D_A'), ('2vir_B_A_C', '6n5e_G_F_B'), ('2vir_B_A_C', '6pdx_M_N_B'), ('4fp8_I_M_B', '1qfu_H_L_A'), ('4fp8_I_M_B', '2vir_B_A_C'), ('4fp8_I_M_B', '4fp8_I_M_B'), ('4fp8_I_M_B', '4gms_J_N_E'), ('4fp8_I_M_B', '4gms_I_M_C'), ('4fp8_I_M_B', '5k9k_H_L_F'), ('4fp8_I_M_B', '5kaq_F_G_C'), ('4fp8_I_M_B', '6n5d_C_D_A'), ('4fp8_I_M_B', '6n5e_G_F_B'), ('4fp8_I_M_B', '6

In [6]:
# Dataset array containing pairs of chain codes
# dataset = [
#     ("4k3j_H_L_B", "4k3j_H_L_B"),
#     ("4ers_H_L_A", "4ers_H_L_A"),
#     ("3tt1_I_M_A", "3tt1_I_M_A"),
#     ("1qfu_H_L_A", "1qfu_H_L_A"),
#     # ...
# ]

In [7]:
# Initialize an empty list to store the extracted sequences
sequences = {}

In [8]:
# Fetch the sequences using the PDB database
for pair in dataset:
    for code in pair:
        if code not in sequences:
            pdb_id, chain_id = code.split("_")[:2]
            seq = get_sequences_from_pdb(pdb_id, chain_id)
            if seq:
                sequences[code] = seq
            else:
                print(f"Invalid sequence for {code}")

/usr/local/lib/python3.10/dist-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 19015.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 19029.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 19081.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain H is discontinuous at line 19161.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain L is discontinuous at line 19180.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain L is discontinuous at line 4901.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain H is discontinuous at line 4928.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 4966.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 28923.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 28961.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 28991.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 29008.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain H is discontinuous at line 29018.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 8014.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 8042.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 8056.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 8084.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain L is discontinuous at line 8114.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain H is discontinuous at line 8117.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 6027.
  warnings.warn(


...

/usr/local/lib/python3.10/dist-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 8197.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 8342.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain C is discontinuous at line 8484.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain D is discontinuous at line 8663.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain E is discontinuous at line 8826.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain F is discontinuous at line 8862.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain C is discontinuous at line 9615.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain H is discontinuous at line 9620.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain L is discontinuous at line 9625.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 9635.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 9678.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain C is discontinuous at line 9729.
  warnings.warn(
/usr/local/lib/python3.10/dist-pac

/usr/local/lib/python3.10/dist-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain H is discontinuous at line 8168.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain L is discontinuous at line 8186.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain C is discontinuous at line 8192.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain H is discontinuous at line 8202.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain L is discontinuous at line 8367.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain C is discontinuous at line 8534.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain Q is discontinuous at line 16200.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain S is discontinuous at line 16206.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain U is discontinuous at line 16212.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain V is discontinuous at line 16218.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain P is discontinuous at line 16224.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain Q is discontinuous at line 16305.
  warnings.warn(
/usr/local/lib/python3.10/di

/usr/local/lib/python3.10/dist-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 9532.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain H is discontinuous at line 9533.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 9534.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 9571.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain H is discontinuous at line 9593.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain L is discontinuous at line 9644.
  warnings.warn(
/usr/local/lib/python3.10/dist-pac

In [10]:
import pandas as pd

# Create a DataFrame from the dataset
df = pd.DataFrame(sequences, columns=["Antibody", "Antigen"])

# Display the DataFrame
print(df)


Empty DataFrame
Columns: [Antibody, Antigen]
Index: []


In [11]:
print(sequences)

{'4k3j_H_L_B': 'EVQLVESGGGLVQPGGSLRLSCAASGYTFTSYWLHWVRQAPGKGLEWVGMIDPSNSDTRFNPNFKDRFTISADTSKNTAYLQMNSLRAEDTAVYYCATYRSYVTPLDYWGQGTLVTVSSASTKGPSVFPLAPSSKSTSGGTAALGCLVKDYFPEPVTVSWNSGALTSGVHTFPAVLQSSGLYSLSSVVTVPSSSLGTQTYICNVNHKPSNTKVDKKVEPKS', '4ers_H_L_A': 'QVQLVESGGGVVQPGRSLRLSCAASGFTFSSYGMHWVRQAPGKGLEWVAVMYYDGSNKDYVDSVKGRFTISRDNSKNTLYLQMNRLRAEDTAVYYCAREKDHYDILTGYNYYYGLDVWGQGTTVTVSSASTKGPSVFPLAPSSKSTSGGTAALGCLVKDYFPEPVTVSWNSGALTSGVHTFPAVLQSSGLYSLSSVVTVPSSSLGTQTYICNVNHKPSNTKVDKKVEPKSC', '3tt1_I_M_A': 'QVQLQQSGAELARPGASVKLSCKASGYTFTDYYINWMKQRTGQGLEWIGEIYPGVGTTYYDEKFKGKATLTADKSSRSAYMQLSSLTSEDSAVYFCARRTVTMGRYAMDYWGQGTSVTVSSAKTTAPSVYPLAPV', '1qfu_H_L_A': 'QVQLQQPGAELVRPGASVKLSCKASGYTLTTYWMNWFKQRPDQGLEWIGRIDPYDSETHYNQKFKDKAILTVDRSSSTAYMQLSSLTSEDSAVYYCTRFLQITTIIYGMDYWGQGTSVTVSSAKTTPPSVYPLAPGSAAQTNSMVTLGCLVKGYFPEPVTVTWNSGSLSSGVHTFPAVLQSDLYTLSSSVTVPSSTWPSETVTCNVAHPASSTKVDKKIVPRD', '2vir_B_A_C': 'QVQLKESGPGLVAPSQSLSITCTVSGFLLISNGVHWVRQPPGKGLEWLGVIWAGGNTNYNSALMSRVSISKDNSKSQVFLKMKSLQTDDTAMYYCARDFYDY

In [12]:
# Imports for model
import torch
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [21]:
# Encode the antibody and antigen sequences
from itertools import chain, product

AA = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']
DP = list(product(AA, AA))
DP_list = []
for i in DP:
    DP_list.append(str(i[0]) + str(i[1]))


def returnCKSAAPcode(query_seq, k):
    code_final = []
    query_seq = query_seq.upper()

    # Check if the sequence length is sufficient
    if len(query_seq) <= k + 1:
        return []
    for turns in range(k + 1):
        DP_dic = {}
        code = []
        code_order = []
        for i in DP_list:
            DP_dic[i] = 0
        for i in range(len(query_seq) - turns - 1):
            tmp_dp_1 = query_seq[i]
            tmp_dp_2 = query_seq[i + turns + 1]
            tmp_dp = tmp_dp_1 + tmp_dp_2
            if tmp_dp in DP_dic.keys():
                DP_dic[tmp_dp] += 1
            else:
                DP_dic[tmp_dp] = 1
        for i, j in DP_dic.items():
            code.append(j / (len(query_seq) - turns - 1))
        for i in DP_list:
            code_order.append(code[DP_list.index(i)])
        # code_final+=code_order
        code_final += code
    return code_final


In [22]:
# Filter the dataset to only include pairs with valid sequences and sufficient length
filtered_dataset = [
    (antibody, antigen)
    for antibody, antigen in dataset
    if antibody in sequences
    and antigen in sequences
    and len(sequences[antibody]) > k_gap + 1
    and len(sequences[antigen]) > k_gap + 1
]

In [23]:
# Encode the filtered dataset
encoded_dataset = []
k_gap = 2
for antibody, antigen in filtered_dataset:
    encoded_antibody = returnCKSAAPcode(sequences[antibody], k_gap)
    encoded_antigen = returnCKSAAPcode(sequences[antigen], k_gap)
    encoded_dataset.append((encoded_antibody, encoded_antigen))

In [24]:
# 4. Split the dataset into training and validation sets
train_set, val_set = train_test_split(encoded_dataset, test_size=0.2, random_state=42)


In [25]:
# Define custom PyTorch Dataset class
class InteractionDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        return (
            torch.tensor(self.dataset[idx][0], dtype=torch.float32),
            torch.tensor(self.dataset[idx][1], dtype=torch.float32)
        )

In [26]:
# THE MODEL!
import torch
import torch.nn as nn

class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)


class SiameseNetwork(nn.Module):
    def __init__(self):
        super(SiameseNetwork, self).__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=10, kernel_size=3, stride=1),
            nn.BatchNorm2d(10),
            nn.LeakyReLU(),
            nn.Conv2d(10, 20, 3, 1),
            nn.BatchNorm2d(20),
            nn.LeakyReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.fc = nn.Sequential(
            nn.Linear(20 * 8 * 8, 500),
            nn.LeakyReLU(),
            nn.Linear(500, 500),
            nn.LeakyReLU(),
            nn.Linear(500, 1)
        )

    def forward_one(self, x):
        x = self.cnn(x)
        x = x.view(x.size()[0], -1)
        x = self.fc(x)
        return x

    def forward(self, x1, x2):
        x1 = x1.view(-1, 1, 20, 20)  # Reshape x1 to (batch_size, 1, 20, 20)
        x2 = x2.view(-1, 1, 20, 20)  # Reshape x2 to (batch_size, 1, 20, 20)
        
        out1 = self.forward_one(x1)
        out2 = self.forward_one(x2)
        return torch.cat((out1, out2), 1)



In [27]:
# 5. Create the Siamese Network model
model = SiameseNetwork()
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [28]:
# 7. Create a DataLoader for the training set
train_loader = DataLoader(InteractionDataset(train_set), batch_size=32, shuffle=True)

In [29]:
# 8. Train the model using the training set
num_epochs = 10
for epoch in range(num_epochs):
    for batch_idx, (antibodies, antigens) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(antibodies, antigens)
        labels = torch.tensor([1 if i % 2 == 0 else 0 for i in range(outputs.size(0))])
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}")

Epoch 1/10, Loss: 0.6935204863548279
Epoch 2/10, Loss: 0.6933976411819458
Epoch 3/10, Loss: 0.6957487463951111
Epoch 4/10, Loss: 0.6904309391975403
Epoch 5/10, Loss: 0.6933752298355103
Epoch 6/10, Loss: 0.6955272555351257
Epoch 7/10, Loss: 0.691971480846405
Epoch 8/10, Loss: 0.694992184638977
Epoch 9/10, Loss: 0.6938749551773071
Epoch 10/10, Loss: 0.6949390769004822


In [30]:
# 9. Evaluate the model using the validation set
val_loader = DataLoader(InteractionDataset(val_set), batch_size=32, shuffle=False)

model.eval()
val_labels_list = []
val_preds_list = []

with torch.no_grad():
    for batch_idx, (antibodies, antigens) in enumerate(val_loader):
        outputs = model(antibodies, antigens)
        batch_labels = [1 if i % 2 == 0 else 0 for i in range(outputs.size(0))]
        val_labels_list.extend(batch_labels)
        
        _, batch_preds = torch.max(outputs, 1)
        val_preds_list.extend(batch_preds.tolist())

In [31]:
# 10. Compute the performance metrics
val_labels = torch.tensor(val_labels_list)
val_preds = torch.tensor(val_preds_list)
accuracy = accuracy_score(val_labels, val_preds)
precision = precision_score(val_labels, val_preds)
recall = recall_score(val_labels, val_preds)
f1 = f1_score(val_labels, val_preds)

print(f"Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1 Score: {f1}")

Accuracy: 0.5114401076716016, Precision: 0.5155875299760192, Recall: 0.38565022421524664, F1 Score: 0.44125192406362235
